## Customer Database Deduplication 

///Objective: Clean a customer database that has duplicates caused by inconsistent data entry//

In [1]:
import pandas as pd
import difflib

///load data:

In [3]:
data = {
    'customer_id': [101, 102, 103, 101, 104],
    'name': [' John Doe ', 'john doe', 'JON DOE', 'John   Doe', 'Alice Smith'],
    'address': [' 12 Main St ', '12 main st', '12 MAIN STREET', ' 12   main st ', '55 Park Ave']
}
df=pd.DataFrame(data)
print("original data is:\n",df)

original data is:
    customer_id         name         address
0          101    John Doe      12 Main St 
1          102     john doe      12 main st
2          103      JON DOE  12 MAIN STREET
3          101   John   Doe   12   main st 
4          104  Alice Smith     55 Park Ave


///Remove dupicates://

In [4]:
df = df.drop_duplicates(subset=['customer_id'], keep='first')

print("\nAfter Removing Exact Duplicates (customer_id):")
print(df)


After Removing Exact Duplicates (customer_id):
   customer_id         name         address
0          101    John Doe      12 Main St 
1          102     john doe      12 main st
2          103      JON DOE  12 MAIN STREET
4          104  Alice Smith     55 Park Ave


//Standardize Text Columns//

In [5]:
df['name'] = df['name'].str.strip().str.upper()
df['address'] = df['address'].str.strip().str.upper()

print("\nAfter Standardizing Text:")
print(df)


After Standardizing Text:
   customer_id         name         address
0          101     JOHN DOE      12 MAIN ST
1          102     JOHN DOE      12 MAIN ST
2          103      JON DOE  12 MAIN STREET
4          104  ALICE SMITH     55 PARK AVE


///Handle fuzzy Duplicates://

In [6]:
def find_fuzzy_duplicates(col, threshold=0.85):
    """Return pairs of possible fuzzy duplicates."""
    values = col.unique()
    duplicates = []

    for i in range(len(values)):
        for j in range(i + 1, len(values)):
            ratio = difflib.SequenceMatcher(None, values[i], values[j]).ratio()
            if ratio >= threshold:
                duplicates.append((values[i], values[j], ratio))
    return duplicates

fuzzy_name_matches = find_fuzzy_duplicates(df['name'])

print("\nPossible Fuzzy Duplicate Names:")
for name1, name2, score in fuzzy_name_matches:
    print(f"{name1}  <-->  {name2} | Similarity Score: {score:.2f}")


Possible Fuzzy Duplicate Names:
JOHN DOE  <-->  JON DOE | Similarity Score: 0.93
